# СОДЕРЖАНИЕ

Образец запроса с описательной информацией
``` python
from sklearn.mixture import GaussianMixture
help(GaussianMixture)
```

## ***РАЗДЕЛЕНИЕ ВЫБОРОК И ВАЛИДАЦИЯ***
* Деление на 3 набора:
    * Тренировочный X_train, y_train (70-80%)
    * Валидационный X_valid, y_valid (10-15%)
    * Тестовый      X_test,  y_test  (10-15%)

* ### HOLD-OUT - "ОТЛОЖЕННАЯ ВЫБОРКА" 
    * Очень простой и понятный
    * Чаще всего применяется на больших датасетов, так как менне ресурсный
    * Можно улучшить стратификацией `stratify=y`
    ``` python
    from sklearn.model_selection import train_test_split
    #разбиваем исходную выборку на тренировочную и валидационную в соотношении 80/20
    X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size=0.2)
    #разбиваем валидационную выборку на валидационную и тестовую в соотношении 50/50
    X_valid, X_test, y_valid, y_test = model_selection.train_test_split(X_valid, y_valid, test_size=0.5)
    ```

* ### K-FOLD - "КРОСС-ВАЛИДАЦИЯ ИЛИ ПЕРЕКРЁСТНЫЙ КОНТРОЛЬ"
    * Исходые данные разбивают на k-фолдов (частей) с отделением тестовых данных
    * Циклично итерируют фолды, при этом один из них по очереди является валидационным
    * Получаем более точный модели, менее чувствительные к выбросам
    * Более ресурсный и медленный, так как число итераций зависит от числа фолдов
    * Если количество фолдов больше 30, можно построить доверительный интервал для среднего значения метрики
    ``` python
    from sklearn import model_selection
    kf = model_selection.KFold() # Обычное разбиение
    skf = model_selection.StratifiedKFold() # Стратифицированное разбиение
    #Считаем метрики на кросс-валидации k-fold
    cv_metrics = model_selection.cross_validate()
    print('Train k-fold mean accuracy: {:.2f}'.format(np.mean(cv_metrics['train_score'])))
    ```

* ### LIVE-ONE-OUT - "ОТЛОЖЕННЫЙ ПРИМЕР ИЛИ ПОЭЛЕМЕНТНАЯ КРОСС-ВАЛИДАЦИЯ"
    * Алгоритм:
        * Повторять n раз:
            * Выбрать один случайный пример для валидации
            * Обучить модель на всех оставшихся  примерах
            * Произвести оценку качества (вычислить метрику) на отложенном примере
        * Усреднить значение метрик на всех примерах
    * Имеет наиболее объективные и надёжные метрики
    * Идеален для небольших данных (порядка 100)
    * Очень ресурсозатратный
    ``` python
    from sklearn import model_selection
    loo = model_selection.LeaveOneOut()
    cv_metrics = model_selection.cross_validate()
    print('Train k-fold mean accuracy: {:.2f}'.format(np.mean(cv_metrics['train_score'])))
    ```
    * Примечание. Метод leave-one-out можно реализовать и без использования специального класса — достаточно просто указать параметр n_split=n в инициализаторе KFold, где n — количество строк в таблице.

* ### БОРЬБА С ДИСБАЛАНСОМ
    * Взвешивание объектов. В функцию ошибки добавляется штраф, прямо пропорциональный количеству объектов каждого класса. 
    * Выбор порога вероятности. Заключается в том, что мы подбираем такой порог вероятности (по умолчанию он равен 0.5 во всех моделях), при котором на валидационной выборке максимизируется целевая метрика.
    * Сэмплирование (sampling) — перебалансировка выборки искусственным путём:  
        * oversampling — искусственное увеличение количества объектов миноритарного класса;
        * undersampling — сокращение количества объектов мажоритарного класса:
            Здесь могут использоваться алгоритмы генерации искусственных данных, такие как NearMiss, SMOTE (Synthetic Minority Oversampling Techniques) и ADASYN (Adaptive Synthetic).
            Мы рассмотрим наиболее популярный алгоритм — SMOTE
        ``` python
        from imblearn.over_sampling import SMOTE
        sm = SMOTE(random_state=42)
        X_train_s, y_train_s = sm.fit_resample(X_train, y_train)
        ```
---

## ***РЕГРЕССИЯ***

* ### АНАЛИТИЧЕСКОЕ РЕШЕНИЕ (NumPy)

* ### АНАЛИТИЧЕСКОЕ РЕШЕНИЕ "Линейная регрессия" (sklearn)
    ``` python
    from sklearn import linear_model    
    lr_model = linear_model.LinearRegression()
    ```

* ### ЧИСЛОВОЕ РЕШЕНИЕ "Градиентный спуск (Gradient descent)" (sklearn)
    ``` python
    from sklearn import linear_model
    sgd_lr_model = linear_model.SGDRegressor()
    ```

* ### ПОЛИНОМИАЛЬНЫЕ ПРИЗНАКИ (Полиномиальная регрессия (Polynomial Regression))
    * Нужно проводить стандартизацию и нормальзацию до этого этапа
    ``` python
    from sklearn import preprocessing
    poly = preprocessing.PolynomialFeatures(degree=2, include_bias=False)
    poly.fit(X_train)
    X_train_poly = poly.transform(X_train)
    ```
    
* ### РЕГУЛЯРИЗАЦИЯ (борьба с разбросом (переобучением)) 
    * L1-регуляризация (Lasso)
        ``` python
        lasso_lr_poly = linear_model.Lasso(alpha=0.1)
        lasso_lr_poly.fit(X_train_scaled_poly, y_train)
        ```
    * L2-регуляризация (Ridge), или регуляризация Тихонова
        ``` python
        ridge_lr_poly = linear_model.Ridge(alpha=10)
        ridge_lr_poly.fit(X_train_scaled_poly, y_train)
        ```

* ### МЕТРИКИ РЕГРЕССИИ 
    ``` python
    from sklearn import metrics
    ```
    * MAE (Mean Absolute Error)             - mean_absolute_error(y, y_pred)
    * MAPE (Mean Absolute Percent Error)    - mean_absolute_percentage_error(y, y_pred)*100
    * MSE (Mean Square Error)               - mean_square_error(y, y_pred)
    * RMSE (Root Mean Squared Error)        - np.sqrt(mean_square_error(y, y_pred))
    * R2 (Коэффициент детерминации)         - r2_score(y, y_pred)

---

## ***КЛАССИФИКАЦИЯ***

* ### ЧИСЛОВОЕ РЕШЕНИЕ "Логистическая регрессия"
    ``` python
    from sklearn import linear_model
    log_reg_model = linear_model.LogisticRegression()
    ```

* ### МУЛЬТИКЛАССОВАЯ КЛАССИФИКАЦИЯ (когда не 2 класса, а больше)
    * Сравнивает класс 0 с классом 1 и 2, потом класс 1 с классами 0 и 2 и т. д.
    ``` python
    from sklearn import linear_model
    log_reg_model = linear_model.LogisticRegression(
        multi_class='multinomial', #мультиклассовая классификация
    )
    ```

* ### ДЕРЕВЬЯ РЕШЕНИЙ (связанный ациклический граф)
    * состоит из корневой вершины, внутренних вершин и листьев
    * Предикаты - критерии находящиеся на вершинах
    ``` python
    from sklearn import tree # Модели деревьев решения
    # Создаём объект класса DecisionTreeClassifier
    dt_clf_model = tree.DecisionTreeClassifier()
    dt_clf_model.get_depth() # Показывает дерево
    ```

* ### АНСАМБЛИ (БЕГГИНГ) "Случайный лес"
    * Много слабых моделей объединяются в одну
    * Виды ансамблей 
        * Бэггинг — параллельно обучаем множество одинаковых моделей, а для предсказания берём среднее по предсказаниям каждой из моделей.
        * Бустинг — последовательно обучаем множество одинаковых моделей, где каждая новая модель концентрируется на тех примерах, где предыдущая допустила ошибку.
        * Стекинг — параллельно обучаем множество разных моделей, отправляем их результаты в финальную модель, и уже она принимает решение.
    ``` python 
    from sklearn import ensemble
    rf_clf_model = ensemble.RandomForestClassifier(
        n_estimators=500, #число деревьев
    )
    ```

* ### МЕТРИКИ КЛАССИФИКАЦИИ 
    ```python
    from sklearn import metrics
    ```
    * Матрица ошибок                        - confusion_matrix(y, y_pred)
    * Accuracy (достоверность)              - accuracy_score(y, y_pred)
        * чем ближе к 1, тем больше угадала
    * Precision (точность) или              - precision_score(y, y_pred)
        * PPV (Positive Predictive Value)
        * чем ближе к 1, тем меньше ошибок
    * Recall (полнота) или                  - recall_score(y, y_pred)
        * TPR (True Positive Rate)
        * чем ближе к 1, тем больше значений 1 класса названо правильно
    * F-мера                                - f1_score(y, y_pred)
        * (это взвешенное среднее гармоническое между precision и recall)
        * чем ближе к 1, тем точнее модель
    * Все ошибки в одном отчете             - classification_report(y, y_pred)

---

## ***КЛАСТЕРИЗАЦИЯ***

* ### ОПРЕДЕЛЕНИЕ ОПТИМАЛЬНОГО КОЛИЧЕСТВА КЛАСТЕРОВ
    * #### МЕТОД ЛОКТЯ - построение графика зависимости инерции от количества кластеров
        ``` python
        model.inertia_ # получение инерции для создания графика
        ```

    * #### КОЭФФИЦИЕНТ СИЛУЭТА - построение графика зависимости коэффициента силуэта от количества кластеров
        * Коэффициент силуэта показывает, насколько объект похож на объекты кластера, в котором он находится, по сравнению с объектами из других кластеров.
        ``` python
        silhouette_score(X, model.labels_)
        ```
* ### EM - АЛГОРИТМЫ КЛАСТЕРИЗАЦИИ
    * В основе данного подхода лежит предположение, что любой объект принадлежит ко всем кластерам, но с разной вероятностью.
    * Представителями являются:
        * K-means кластеризация - например, для кластеризации документов
        * GMM кластеризация - например, для сегментации изображения


* ### КЛАСТЕРИЗАЦИЯ "АЛГОРИТМ K-MEANS"
    * Чувствительна к выбросам
    * K-MEANS - центроиды кластера это средние значения
    * K-MEANS++ - центроиды кластера это средние значения, но первые значения выбираются не случайно
    * K-MEDIANS - центроиды кластера это медианные значения
    * K-MEDOIDS - центроиды кластера это медианные значения, но не расчетное значение, а ближайшая к нему точка
    * FUZZY C-MEANS - каждый объект может принадлежать к разным кластерам с разной вероятностью
    ``` python
    from sklearn.cluster import KMeans
    k_means_model = KMeans(n_clusters=2, init='k-means++', n_init=10, random_state=42)
    ```
* ### КЛАСТЕРИЗАЦИЯ "GMM" - модель гауссовой смеси (Gaussian Mixture Model)
    * Чувствительна к выбросам
    ``` python
    from sklearn.mixture import GaussianMixture
    gm_clst_model = GaussianMixture()
    ```

* ### "ИЕРАРХИЧЕСКАЯ" КЛАСТЕРИЗАЦИЯ
    * Принцип иерархической кластеризации основан на построении дерева (иерархии) вложенных кластеров.
    * Строится дендрограмма - это древовидная диаграмма, которая содержит  уровней. Каждый уровень — это шаг укрупнения кластеров.
    * Методы построения дендрограммы:
        * Агломеративный метод (agglomerative) - объединение мелких кластеров
        * Дивизионный (дивизивный) метод (divisive) - деление крупных кластеров
    * Методы расчета расстояния между кластерами
        * Метод одиночной связи (min расстояние)
        * Метод полной связи    (max расстояние)
        * Метод средней связи   (mean расстояние)
        * Центроидный метод     (расстояние м/ж центрами)
    ``` python
    from sklearn.cluster import AgglomerativeClustering
    agg_clst_model = AgglomerativeClustering()
    ```

* ### **ПОНИЖЕНИЕ РАЗМЕРНОСТИ**
    * #### СПЕКТРАЛЬНАЯ КЛАСТЕРИЗАЦИЯ
        * 
        * Применяется для: 
            * сегментации изображений
        ``` python
        from sklearn.cluster import SpectralClustering
        spectral_clst_model = SpectralClustering()
        ```

    * #### PCA - линейное преобразование
        * Метод главных компонент, или PCA (Principal Components Analysis)
        * это один из базовых способов уменьшения размерности
        * Применяется для:  
            * подавление шума
            * индексация видео
        ``` python
        from sklearn.decomposition import PCA
        pca = PCA()
        ```

    * #### t-SNE - нелинейное преобразование
        * t-SNE (t-distributed Stochastic Neighbor Embedding)
        * «стохастическое вложение соседей с t-распределением»
        * при преобразовании похожие объекты оказываются рядом, а непохожие — далеко друг от друга
        * Применяется для:  
            * уменьшения размерность до 2х или 3х мерного
        ``` python
        from sklearn.manifold import TSNE
        tsne = TSNE()
        ```

* ### КЛАСТЕРИЗАЦИЯ НА ОСНОВЕПЛОТНОСТИ (DBSCAN)
    * DENSITY-BASED SPATIAL CLUSTERING OF APPLICATIONS WITH NOISE
    * В отличие от k-means, не нужно задавать количество кластеров — алгоритм сам определит оптимальное
    * Алгоритм хорошо работает с данными произвольной формы
    * DBSCAN отлично справляется с выбросами в датасетах
    ``` python
    from sklearn.cluster import DBSCAN
    clst_model = DBSCAN()    
    ```
* ### ВИЗУАЛИЗАЦИЯ КЛАСТЕРИЗАЦИЙ
    * диаграмма рассеяния для двухмерного и трёхмерного случаев 
    * Convex Hull, или выпуклая оболочка - провести гарницы
    * дендрограмма 
    * Clustergram - "полосы" - только для иерархических кластеризаций

* ### МЕТРИКИ КЛАСТЕРИЗАЦИИ 
    ``` python
    from sklearn.metrics.cluster import homogeneity_score
    from sklearn.metrics.cluster import completeness_score
    from sklearn.metrics.cluster import v_measure_score
    from sklearn.metrics.cluster import rand_score
    ```
    * Однородность кластеров                - homogeneity_score(y, y_pred)
    * Полнота кластеров                     - completeness_score(y, y_pred)
    * V-мера (комбинация однор. и полн.)    - v_measure_score(y, y_pred)
    * Индекс Ренда                          - rand_score(y, y_pred)

---



---
---
---

In [ ]:
# Базовые библиотеки
import numpy as np # для матричных вычислений
import pandas as pd # для анализа и предобработки данных
import matplotlib.pyplot as plt # для визуализации
import seaborn as sns # для визуализации

# Модели
from sklearn import linear_model # линейные модели
from sklearn import tree # деревья
from sklearn.cluster import KMeans # KMeans кластеризация
from sklearn.mixture import GaussianMixture # GMM кластеризация
from sklearn.cluster import SpectralClustering # Спектральная кластеризация
from sklearn.decomposition import PCA # PCA снижение размерности
from sklearn.manifold import TSNE # T-SNE снижение размерности
from sklearn.cluster import DBSCAN # DBSCAN кластеризация

# Метрики регрессии и классификации
from sklearn import metrics # метрики 

# Метрики кластеризации
from sklearn.metrics.cluster import homogeneity_score
from sklearn.metrics.cluster import completeness_score
from sklearn.metrics.cluster import v_measure_score
from sklearn.metrics.cluster import rand_score


# Работа с данными для моделей
from sklearn.model_selection import train_test_split # сплитование выборки
from sklearn import model_selection # для K-Fold и LIVE-ONE-OUT
from sklearn import ensemble # ансамбли

#from imblearn.over_sampling import SMOTE # Для сэмплирования (придумывания) данных
from sklearn import preprocessing # предобработка
%matplotlib inline
plt.style.use('seaborn')

In [ ]:
data = pd.read_csv('pokemon.csv')
data.head()

In [ ]:
# матрица наблюдений и вектор правильных ответов
X, y = data.drop('RealClusters', axis=1), data['RealClusters'] 

## ***РАЗДЕЛЕНИЕ ВЫБОРОК И ВАЛИДАЦИЯ***

## HOLD-OUT - "ОТЛОЖЕННАЯ ВЫБОРКА"
* Деление на 3 набора:
    * Тренировочный X_train, y_train (70-80%)
    * Валидационный X_valid, y_valid (10-15%)
    * Тестовый      X_test,  y_test  (10-15%)
* Очень простой и понятный
* Чаще всего применяется на больших датасетов, так как менне ресурсный
* Можно улучшить стратификацией `stratify=y`

In [ ]:

from sklearn.model_selection import train_test_split

# Разделяем выборку на тренировочную и валидационную в соотношении 70/30 со стратификацией
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

# Разделяем выборку на валидационную и тестовую в соотношении 50/50 со стратификацией
X_valid, X_test, y_valid, y_test = train_test_split(X_valid, y_valid, test_size=0.5, stratify=y_valid, random_state=42)

# Выводим результирующие размеры таблиц
print('Train:', X_train.shape, y_train.shape)
print('Valid:', X_valid.shape, y_valid.shape)
print('Test:', X_test.shape, y_test.shape)

# train_test_split(
    # X - матрица признаков (наблюдений)
    # y - вектор правильных ответов
    # test_size=0.3 - 30% данных идут в тестовую выборку
    # train_size - размер тренировочной выборки. Может быть указан в долях. 
        # Определяется автоматически, если параметр test_size передан как 1-test_size.    
    # shuffle=False - По умолчанию True. Параметр перемешивания данных в выборке.
    # stratify=y - Стратифицированное разбиение - Одинаковое соотношение целевого признака 
        # в тренировочной и тестовой выборке, не допускающее перекоса в обучении модели.
    # random_state=42 - гарантирует выдачу генератором одних и тех же случайных значений
# )

## K-FOLD - "КРОСС-ВАЛИДАЦИЯ ИЛИ ПЕРЕКРЁСТНЫЙ КОНТРОЛЬ"
* Исходые данные разбивают на k-фолдов (частей) с отделением тестовых данных
* Циклично итерируют фолды, при этом один из них по очереди является валидационным
* Получаем более точный модели, менее чувствительные к выбросам
* Более ресурсный и медленный, так как число итераций зависит от числа фолдов
* Если количество фолдов больше 30, можно построить доверительный интервал для среднего значения метрики

In [ ]:
from sklearn import model_selection

# Создаём кросс-валидатор KFold
kf = model_selection.KFold(n_splits=5) # Обычное разбиение
skf = model_selection.StratifiedKFold(n_splits=5) # Стратифицированное разбиение

# Создаём кросс-валидатор LeaveOneOut
loo = model_selection.LeaveOneOut()
 
#Считаем метрики на кросс-валидации k-fold
cv_metrics = model_selection.cross_validate(
    estimator=model, #модель
    X=X, #матрица наблюдений X
    y=y, #вектор ответов y
    cv=kf, #кросс-валидатор KFold
    #cv=skf, #кросс-валидатор SKFold
    #cv=loo, #кросс-валидатор LeaveOneOut
    scoring='accuracy', #метрика
    return_train_score=True #подсчёт метрики на тренировочных фолдах
)

print('Train k-fold mean f1: {:.2f}'.format(np.mean(cv_metrics['train_score'])))
print('Valid k-fold mean f1: {:.2f}'.format(np.mean(cv_metrics['test_score'])))

#Делаем предсказание вероятностей на кросс-валидации
y_cv_proba_pred = model_selection.cross_val_predict(model, X_train, y_train, cv=skf, method='predict_proba')

#Выделяем столбец с вероятностями для класса 0
y_cv_proba_pred_0 = y_cv_proba_pred[:, 0]

#Выделяем столбец с вероятностями для класса 1 
y_cv_proba_pred_1 = y_cv_proba_pred[:, 1]

# KFold(
    # n_splits=5 - число фолдов (частей)
# )

## ВЫБОР ПОРОГА ВЕРОЯТНОСТИ. PR-КРИВАЯ.
PR-кривая (precision-recall curve) — это график зависимости precision от recall при различных значениях порога вероятности.

Мы можем построить PR-кривую. Для этого воспользуемся функций precision_recall_curve() из модуля metrics библиотеки sklearn. В данную функцию нужно передать истинные метки классов и предсказанные вероятности. Взамен она вернёт три массива: значения метрик precision и recall, вычисленных на различных порогах вероятности, и сами пороги вероятности:

In [ ]:
#Вычисляем координаты PR-кривой
precision, recall, thresholds = metrics.precision_recall_curve(y_train, y_cv_proba_pred)

#Вычисляем F1-score при различных threshold
f1_scores = (2 * precision * recall) / (precision + recall)
#Определяем индекс максимума
idx = np.argmax(f1_scores)
print('Best threshold = {:.2f}, F1-Score = {:.2f}'.format(thresholds[idx], f1_scores[idx]))
 
#Строим PR-кривую
fig, ax = plt.subplots(figsize=(10, 5)) #фигура + координатная плоскость
#Строим линейный график зависимости precision от recall
ax.plot(precision, recall, label='Decision Tree PR')
#Отмечаем точку максимума F1
ax.scatter(precision[idx], recall[idx], marker='o', color='black', label='Best F1 score')
#Даём графику название и подписываем оси
ax.set_title('Precision-recall curve')
ax.set_xlabel('Recall')
ax.set_ylabel('Precision')
#Отображаем легенду
ax.legend();    

Сделаем предсказание классов с таким порогом для всех объектов из отложенной валидационной выборки и выведем отчёт о метриках:

In [ ]:
#Образцы воды, для которых вероятность быть пригодными для питья > threshold_opt, относим к классу 1
#В противном случае — к классу 0
y_valid_pred_proba = model.predict_proba(X_valid)[:, 1]
y_valid_pred = (y_valid_pred_proba > threshold_opt).astype('int')
#Считаем метрики
print(metrics.classification_report(y_valid, y_valid_pred))

## СЭМПЛИРОВАНИЕ

Следующий подход работы в условиях дисбаланса классов, который мы рассмотрим, — сэмплирование, а точнее — пересэмплирование (oversampling).

Идея очень проста: если у нас мало наблюдений миноритарного класса, следует искусственно увеличить их количество.

Для пользователей pip:

`!pip install imbalanced-learn`
Для пользователей anaconda:

`!conda install -c conda-forge imbalanced-learn`
Все алгоритмы пересэмплирования находятся в модуле over_sampling библиотеки imblearn. Импортируем оттуда алгоритм SMOTE:

Создадим объект класса SMOTE и вызовем у него метод fit_sample(), передав в него обучающую выборку (X_train, y_train). Затем выведем количество наблюдений каждого из классов до и после сэмплирования:

### ТОЛЬКО ДЛЯ ТРЕНИРОВОЧНОЙ ВЫБОРКИ!!!

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
X_train_s, y_train_s = sm.fit_resample(X_train, y_train)

print('Train shape before oversampling:', X_train.shape) 
print('Class balance before oversampling: \n', y_train.value_counts(), sep='')
print('-'*40)
print('Train shape after oversampling:', X_train_s.shape)
print('Class balance after oversampling: \n', y_train_s.value_counts(), sep='')


## КРИВАЯ ОБУЧЕНИЯ (learning curve) 
* это график зависимости некоторой метрики на обучающем (валидационном) наборе данных от количества объектов, которые участвуют в обучении модели.

In [ ]:
#Создаём объект кросс-валидатора k-fold со стратификацией
skf = model_selection.StratifiedKFold(n_splits=5)
 
#Вычисляем координаты для построения кривой обучения
train_sizes, train_scores, valid_scores = model_selection.learning_curve(
    estimator = model, #модель
    X = X, #матрица наблюдений X
    y = y, #вектор ответов y
    cv = skf, #кросс-валидатор
    scoring = 'f1' #метрика
)


In [ ]:
def plot_learning_curve(model, X, y, cv, scoring="f1", ax=None, title=""):
    # Вычисляем координаты для построения кривой обучения
    train_sizes, train_scores, valid_scores = model_selection.learning_curve(
        estimator=model,  # модель
        X=X,  # матрица наблюдений X
        y=y,  # вектор ответов y
        cv=cv,  # кросс-валидатор
        scoring=scoring,  # метрика
    )
    # Вычисляем среднее значение по фолдам для каждого набора данных
    train_scores_mean = np.mean(train_scores, axis=1)
    valid_scores_mean = np.mean(valid_scores, axis=1)
    # Если координатной плоскости не было передано, создаём новую
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 4))  # фигура + координатная плоскость
    # Строим кривую обучения по метрикам на тренировочных фолдах
    ax.plot(train_sizes, train_scores_mean, label="Train")
    # Строим кривую обучения по метрикам на валидационных фолдах
    ax.plot(train_sizes, valid_scores_mean, label="Valid")
    # Даём название графику и подписи осям
    ax.set_title("Learning curve: {}".format(title))
    ax.set_xlabel("Train data size")
    ax.set_ylabel("Score")
    # Устанавливаем отметки по оси абсцисс
    ax.xaxis.set_ticks(train_sizes)
    # Устанавливаем диапазон оси ординат
    ax.set_ylim(0, 1)
    # Отображаем легенду
    ax.legend()
    
# plot_learning_curve(
    # estimator — модель, качество которой будет проверяться на кросс-валидации.
    # X — матрица наблюдений.
    # y — вектор-столбец правильных ответов.
    # train_sizes — относительное (долевое) или абсолютное количество обучающих примеров, которые будут 
        # использоваться для создания кривой обучения. Если dtype имеет значение float, он 
        # рассматривается как часть максимального размера обучающего набора (который определяется выбранным методом проверки), 
        # т. е. он должен быть в пределах (0, 1].
        # По умолчанию используется список [0.1, 0.325, 0.55, 0.775, 1.0], то есть для построения кривой обучения 
        # используется пять точек. Первая точка кривой обучения строится по 10 % наблюдений из обучающего набора, 
        # вторая точка — по 32.5 % и так далее до тех пор, пока в построении модели не будет участвовать весь обучающий набор данных.
    # cv — кросс-валидатор из библиотеки sklearn (например, KFold) или количество фолдов, на которые необходимо разбить выборку. 
        # По умолчанию используется кросс-валидация k-fold на пяти фолдах.
    # scoring — название метрики в виде строки либо функция для её вычисления.
# )

Можно сравниавть метрики разных моделей, загрузив их в список `models`.
``` python
    models = [
        model_1,
        ...,
        model_n
    ]
```

In [ ]:
#Создаём объект кросс-валидатора k-fold со стратификацией
skf = model_selection.StratifiedKFold(n_splits=5)
#Визуализируем кривые обучения
fig, axes = plt.subplots(1, 3, figsize=(15, 4)) #фигура + три координатных плоскости
#Создаём цикл по списку моделей и индексам этого списка
for i, model in enumerate(models): #i — текущий индекс, model — текущая модель
    plot_learning_curve(model, X, y, skf, ax=axes[i], title=f'model {i+1}')

## ***РЕГРЕССИЯ***

### АНАЛИТИЧЕСКОЕ РЕШЕНИЕ (NumPy)

In [ ]:
def linear_regression(X, y):
    # Создаём вектор из единиц
    ones = np.ones(X.shape[0])
    # Добавляем вектор к таблице первым столбцом
    X = np.column_stack([ones, X])
    # Вычисляем обратную матрицу Q
    Q = np.linalg.inv(X.T @ X)
    # Вычисляем вектор коэффициентов
    w = Q @ X.T @ y
    return w
# Вычисляем параметры линейной регрессии
w = linear_regression(X, y)
# Выводим вычисленные значения параметров в виде вектора
print('Vector w: {}'.format(w))
# Выводим параметры с точностью до двух знаков после запятой
print('w0 = {:.2f}'.format(w[0]))
print('w1 = {:.2f}'.format(w[1]))

### АНАЛИТИЧЕСКОЕ РЕШЕНИЕ "Линейная регрессия" (sklearn)

In [ ]:
# Создаём объект класса LinearRegression
lr_model = linear_model.LinearRegression()

# Обучаем модель — ищем параметры по МНК
lr_model.fit(X, y) 

# Выводим полученные параметры
print('w0 = {}'.format(lr_model.intercept_)) #свободный член w0
print('w1 = {}'.format(lr_model.coef_)) #остальные параметры модели w1, w2, ..., wm

# Предсказываем медианную цену для всех участков из набора данных
y_predict = lr_model.predict(X)

# Выводим предсказание
# Составляем таблицу из признаков и их коэффициентов
w_df = pd.DataFrame({'Features': X.columns, 'Coefficients': lr_model.coef_})
# Составляем строку таблицы со свободным членом
intercept_df =pd.DataFrame({'Features': ['INTERCEPT'], 'Coefficients': lr_model.intercept_})
coef_df = pd.concat([w_df, intercept_df], ignore_index=True)
display(coef_df)

### ЧИСЛОВОЕ РЕШЕНИЕ "Градиентный спуск (Gradient descent)" (sklearn)

Часто требуется стандартизация данных, для коррекции масштаба. Предсказания на валидационных и тестовых выборках нужно делать так же после стандартизации.

In [ ]:
from sklearn import preprocessing
 
# Инициализируем стандартизатор StandardScaler
scaler = preprocessing.StandardScaler()

# Подгоняем параметры стандартизатора (вычисляем среднее и СКО)
scaler.fit(X_train)

# Производим стандартизацию тренировочной выборки
X_train_scaled = scaler.transform(X_train)

# Производим стандартизацию тестовой выборки
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Создаём объект класса линейной регрессии с SGD
sgd_lr_model = linear_model.SGDRegressor(random_state=42)

# Обучаем модель — ищем параметры по методу SGD
sgd_lr_model.fit(X, y)

# Выводим полученные параметры
print('w0: {}'.format(sgd_lr_model.intercept_)) #свободный член w0
print('w1: {}'.format(sgd_lr_model.coef_)) #остальные параметры модели w1, w2, ..., wm

#Предсказываем медианную цену для всех участков из набора данных
y_predict = sgd_lr_model.predict(X)

# Выводим предсказание
# Составляем таблицу из признаков и их коэффициентов
w_df = pd.DataFrame({'Features': X.columns, 'Coefficients': sgd_lr_model.coef_})
# Составляем строку таблицы со свободным членом
intercept_df =pd.DataFrame({'Features': ['INTERCEPT'], 'Coefficients': sgd_lr_model.intercept_})
coef_df = pd.concat([w_df, intercept_df], ignore_index=True)
display(coef_df)

# SGDRegressor(
    # loss — функция потерь. По умолчанию используется squared_loss — уже привычная нам MSE. 
        # Но могут использоваться и несколько других. Например, значение "huber" определяет функцию потерь Хьюбера. 
        # Эта функция менее чувствительна к наличию выбросов, чем MSE.
    # max_iter — максимальное количество итераций, выделенное на сходимость. Значение по умолчанию — 1000.
    # learning_rate — режим управления темпом обучения. Значение по умолчанию — 'invscaling'. 
        # Этот режим уменьшает темп обучения по формуле, которую мы рассматривали ранее: etat=eta0/t^p.
        # Есть ещё несколько режимов управления, о которых вы можете прочитать в документации.
        # Если вы не хотите, чтобы темп обучения менялся на протяжении всего обучения, 
        # то можете выставить значение параметра на "constant".
    # eta0 — начальное значение темпа обучения . Значение по умолчанию — 0.01.
        # Если параметр learning_rate="constant", то значение этого параметра 
        # будет темпом обучения на протяжении всех итераций.
    # power_t — значение мощности уменьшения  в формуле etat=eta0/t^p . Значение по умолчанию — 0.25.
# )

### ПОЛИНОМИАЛЬНЫЕ ПРИЗНАКИ (Полиномиальная регрессия (Polynomial Regression))
* Нужно проводить стандартизацию и нормальзацию до этого этапа
``` python
    from sklearn import preprocessing
    poly = preprocessing.PolynomialFeatures(degree=2, include_bias=False)
    poly.fit(X_train)
    X_train_poly = poly.transform(X_train)
```
    


Стандартизацию (нормализацию) полезнее проводить перед 
генерацией полиномиальных признаков, иначе можно потерять масштаб полиномов.

In [ ]:
from sklearn import preprocessing
 
# Инициализируем стандартизатор StandardScaler
scaler = preprocessing.StandardScaler()

# Подгоняем параметры стандартизатора (вычисляем среднее и СКО)
scaler.fit(X_train)

# Производим стандартизацию тренировочной выборки
X_train_scaled = scaler.transform(X_train)

# Производим стандартизацию тестовой выборки
X_test_scaled = scaler.transform(X_test)

In [ ]:
#Создаём генератор полиномиальных признаков
poly = preprocessing.PolynomialFeatures(degree=2, include_bias=False)
poly.fit(X_train_scaled)
#Генерируем полиномиальные признаки для тренировочной выборки
X_train_scaled_poly = poly.transform(X_train_scaled)
#Генерируем полиномиальные признаки для тестовой выборки
X_test_scaled_poly = poly.transform(X_test_scaled)
#Выводим результирующие размерности таблиц
print(X_train_scaled_poly.shape)
print(X_test_scaled_poly.shape)

# PolynomialFeatures(
    # degree — степень полинома. По умолчанию используется степень 2
    # include_bias — включать ли в результирующую таблицу столбец из единиц (x в степени 0). 
        # По умолчанию стоит True, но лучше выставить его в значение False, так как столбец из единиц и 
        # так добавляется в методе наименьших квадратов.
# )

### РЕГУЛЯРИЗАЦИЯ (борьба с разбросом (переобучением)) 

Недообучение (underfitting) — проблема, обратная переобучению. Модель из-за своей слабости не уловила никаких закономерностей в данных. В этом случае ошибка будет высокой как для тренировочных данных, так и для данных, не показанных во время обучения.

Смещение (bias) — это математическое ожидание разности между истинным ответом и ответом, выданным моделью. То есть это ожидаемая ошибка модели.

Разброс (variance) — это вариативность ошибки, то, насколько ошибка будет отличаться, если обучать модель на разных наборах данных. Математически это дисперсия (разброс) ответов модели.

Регуляризация — способ уменьшения переобучения моделей машинного обучения.

Штраф — это дополнительное неотрицательное слагаемое в выражении для функции потерь, которое специально повышает ошибку.  За счёт этого слагаемого метод оптимизации (OLS или SGD) будет находить не истинный минимум функции потерь, а псевдоминимум.

In [ ]:
# L1-регуляризация (Lasso) 
# Создаём объект класса линейной регрессии с L1-регуляризацией
lasso_lr_poly = linear_model.Lasso(alpha=0.1)

# Обучаем модель
lasso_lr_poly.fit(X_train_scaled_poly, y_train)

# Делаем предсказание для тренировочной выборки
y_train_predict_poly = lasso_lr_poly.predict(X_train_scaled_poly)
# Делаем предсказание для тестовой выборки
y_test_predict_poly = lasso_lr_poly.predict(X_test_scaled_poly)

# Рассчитываем коэффициент детерминации для двух выборок
print("Train R^2: {:.3f}".format(metrics.r2_score(y_train, y_train_predict_poly)))
print("Test R^2: {:.3f}".format(metrics.r2_score(y_test, y_test_predict_poly)))

# Lasso(
    # Главный параметр инициализации Lasso — это alpha, коэффициент регуляризации. 
        # По умолчанию alpha=1. Практика показывает, что это довольно сильная регуляризация для L1-метода. 
        # Давайте установим значение этого параметра на 0.1.
# )

In [ ]:
# L2-регуляризация (Ridge)
# Создаём объект класса линейной регрессии с L2-регуляризацией
ridge_lr_poly = linear_model.Ridge(alpha=10)

# Обучаем модель
ridge_lr_poly.fit(X_train_scaled_poly, y_train)

# Делаем предсказание для тренировочной выборки
y_train_predict_poly = ridge_lr_poly.predict(X_train_scaled_poly)
# Делаем предсказание для тестовой выборки
y_test_predict_poly = ridge_lr_poly.predict(X_test_scaled_poly)

# Рассчитываем коэффициент детерминации для двух выборок
print("Train R^2: {:.3f}".format(metrics.r2_score(y_train, y_train_predict_poly)))
print("Test R^2: {:.3f}".format(metrics.r2_score(y_test, y_test_predict_poly)))

# Ridge(
    # Главный параметр инициализации Lasso — это alpha, коэффициент регуляризации. 
        # Для L2-регуляризации параметр alpha по умолчанию равен 1. 
        # Давайте попробуем использовать значение параметра alpha=10:
# )

ЛУЧШЕ ТА, У КОТОРОЙ ВЫШЕ ПОКАЗАТЕЛИ НА ТЕСТОВОЙ ВЫБОРКЕ

Параметр alpha имеет очень важное значение: от его выбора зависит, как сильно мы будем штрафовать модель за переобучение. Важно найти значение, которое приносит наилучший эффект.

Попробуйте вручную изменять параметр alpha для построенных ранее моделей. Согласитесь, это не очень удобно.

In [ ]:
# Создаём список из 20 возможных значений от 0.001 до 1
alpha_list = np.linspace(0.001, 1, 20)
# Создаём пустые списки, в которые будем добавлять результаты 
train_scores = []
test_scores = []
for alpha in alpha_list:
    # Создаём объект класса линейной регрессии с L1-регуляризацией
    lasso_lr_poly = linear_model.Lasso(alpha=alpha, max_iter=10000)
    
    # Обучаем модель
    lasso_lr_poly.fit(X_train_scaled_poly, y_train)
    
    # Делаем предсказание для тренировочной выборки
    y_train_predict_poly = lasso_lr_poly.predict(X_train_scaled_poly)
    # Делаем предсказание для тестовой выборки
    y_test_predict_poly = lasso_lr_poly.predict(X_test_scaled_poly)
    
    # Рассчитываем коэффициенты детерминации для двух выборок и добавляем их в списки
    train_scores.append(metrics.r2_score(y_train, y_train_predict_poly))
    test_scores.append(metrics.r2_score(y_test, y_test_predict_poly))

# Визуализируем изменение R^2 в зависимости от alpha
fig, ax = plt.subplots(figsize=(12, 4)) # фигура + координатная плоскость
ax.plot(alpha_list, train_scores, label='Train') # линейный график для тренировочной выборки
ax.plot(alpha_list, test_scores, label='Test') # линейный график для тестовой выборки
ax.set_xlabel('Alpha') # название оси абсцисс
ax.set_ylabel('R^2') # название оси ординат
ax.set_xticks(alpha_list) # метки по оси абсцисс
ax.xaxis.set_tick_params(rotation=45) # поворот меток на оси абсцисс
ax.legend(); # отображение легенды   

Выбираем на графике лучший результат *alpha*

In [ ]:
# Создаём объект класса линейной регрессии с L1-регуляризацией
lasso_lr_poly = linear_model.Lasso(alpha=0.0536)

# Обучаем модель 
lasso_lr_poly.fit(X_train_scaled_poly, y_train)

# Делаем предсказание для тренировочной выборки
y_train_predict_poly = lasso_lr_poly.predict(X_train_scaled_poly)
# Делаем предсказание для тестовой выборки
y_test_predict_poly = lasso_lr_poly.predict(X_test_scaled_poly)

# Рассчитываем коэффициент детерминации для двух выборок
print("Train R^2: {:.3f}".format(metrics.r2_score(y_train, y_train_predict_poly)))
print("Test R^2: {:.3f}".format(metrics.r2_score(y_test, y_test_predict_poly)))

### МЕТРИКИ РЕГРЕССИИ 
``` python
    from sklearn import metrics
```
* MAE (Mean Absolute Error)             - mean_absolute_error(y, y_pred)
* MAPE (Mean Absolute Percent Error)    - mean_absolute_percentage_error(y, y_pred)*100
* MSE (Mean Square Error)               - mean_square_error(y, y_pred)
* RMSE (Root Mean Squared Error)        - np.sqrt(mean_square_error(y, y_pred))
* R2 (Коэффициент детерминации)         - r2_score(y, y_pred)

In [ ]:
from sklearn import metrics

# Делаем предсказание по всем признакам
y_predict = model.predict(X)
# Рассчитываем MAE - Средняя абсолютная ошибка
print('MAE score: {:.3f} thou. $'.format(metrics.mean_absolute_error(y, y_predict)))
# Рассчитываем MAPE - Средняя абсолютная ошибка в процентах
print('MAPE score: {:.3f} %'.format(metrics.mean_absolute_percentage_error(y, y_predict) * 100))
# Рассчитываем RMSE - Корень из средней квадратической ошибки
print('RMSE score: {:.3f} thou. $'.format(np.sqrt(metrics.mean_squared_error(y, y_predict))))
# Рассчитываем коэффициент детерминации Коэффициент детерминации (R2)
print('R2 score: {:.3f}'.format(metrics.r2_score(y, y_predict)))

---
---
---

## ***КЛАССИФИКАЦИЯ***


### ЧИСЛОВОЕ РЕШЕНИЕ "Логистическая регрессия"
``` python
    from sklearn import linear_model
    log_reg_model = linear_model.LogisticRegression()
```



### МУЛЬТИКЛАССОВАЯ КЛАССИФИКАЦИЯ (когда не 2 класса, а больше)
* Сравнивает класс 0 с классом 1 и 2, потом класс 1 с классами 0 и 2 и т. д.
``` python
    from sklearn import linear_model
    log_reg_model = linear_model.LogisticRegression(
        multi_class='multinomial', #мультиклассовая классификация
    )
```



### ДЕРЕВЬЯ РЕШЕНИЙ (связанный ациклический граф)
* состоит из корневой вершины, внутренних вершин и листьев
* Предикаты - критерии находящиеся на вершинах
``` python
    from sklearn import tree # Модели деревьев решения
    # Создаём объект класса DecisionTreeClassifier
    dt_clf_model = tree.DecisionTreeClassifier()
    dt_clf_model.get_depth() # Показывает дерево
```



### АНСАМБЛИ (БЕГГИНГ) "Случайный лес"
* Много слабых моделей объединяются в одну
* Виды ансамблей 
    * Бэггинг — параллельно обучаем множество одинаковых моделей, а для предсказания берём среднее по предсказаниям каждой из моделей.
    * Бустинг — последовательно обучаем множество одинаковых моделей, где каждая новая модель концентрируется на тех примерах, где предыдущая допустила ошибку.
    * Стекинг — параллельно обучаем множество разных моделей, отправляем их результаты в финальную модель, и уже она принимает решение.
``` python 
    from sklearn import ensemble
    rf_clf_model = ensemble.RandomForestClassifier(
        n_estimators=500, #число деревьев
    )
```

### МЕТРИКИ КЛАССИФИКАЦИИ 
```python
    from sklearn import metrics
```
* Матрица ошибок                        - confusion_matrix(y, y_pred)
* Accuracy (достоверность)              - accuracy_score(y, y_pred)
    * чем ближе к 1, тем больше угадала
* Precision (точность) или              - precision_score(y, y_pred)
    * PPV (Positive Predictive Value)
    * чем ближе к 1, тем меньше ошибок
* Recall (полнота) или                  - recall_score(y, y_pred)
    * TPR (True Positive Rate)
    * чем ближе к 1, тем больше значений 1 класса названо правильно
* F-мера                                - f1_score(y, y_pred)
    * (это взвешенное среднее гармоническое между precision и recall)
    * чем ближе к 1, тем точнее модель
* Все ошибки в одном отчете             - classification_report(y, y_pred)

---
---
---

## ***КЛАСТЕРИЗАЦИЯ***


### ОПРЕДЕЛЕНИЕ ОПТИМАЛЬНОГО КОЛИЧЕСТВА КЛАСТЕРОВ
* #### МЕТОД ЛОКТЯ - построение графика зависимости инерции от количества кластеров
    ``` python
        model.inertia_ # получение инерции для создания графика
    ```

* #### КОЭФФИЦИЕНТ СИЛУЭТА - построение графика зависимости коэффициента силуэта от количества кластеров
    * Коэффициент силуэта показывает, насколько объект похож на объекты кластера, в котором он находится, по сравнению с объектами из других кластеров.
    ``` python
    silhouette_score(X, model.labels_)
    ```


### EM - АЛГОРИТМЫ КЛАСТЕРИЗАЦИИ
* В основе данного подхода лежит предположение, что любой объект принадлежит ко всем кластерам, но с разной вероятностью.
* Представителями являются:
    * K-means кластеризация - например, для кластеризации документов
    * GMM кластеризация - например, для сегментации изображения


### КЛАСТЕРИЗАЦИЯ "АЛГОРИТМ K-MEANS"
* Чувствительна к выбросам
* K-MEANS - центроиды кластера это средние значения
* K-MEANS++ - центроиды кластера это средние значения, но первые значения выбираются не случайно
* K-MEDIANS - центроиды кластера это медианные значения
* K-MEDOIDS - центроиды кластера это медианные значения, но не расчетное значение, а ближайшая к нему точка
* FUZZY C-MEANS - каждый объект может принадлежать к разным кластерам с разной вероятностью
``` python
    from sklearn.cluster import KMeans
    k_means_model = KMeans(n_clusters=2, init='k-means++', n_init=10, random_state=42)
```


### КЛАСТЕРИЗАЦИЯ "GMM" - модель гауссовой смеси (Gaussian Mixture Model)
* Чувствительна к выбросам
``` python
    from sklearn.mixture import GaussianMixture
    gm_clst_model = GaussianMixture()
```


### "ИЕРАРХИЧЕСКАЯ" КЛАСТЕРИЗАЦИЯ
* Принцип иерархической кластеризации основан на построении дерева (иерархии) вложенных кластеров.
* Строится дендрограмма - это древовидная диаграмма, которая содержит  уровней. Каждый уровень — это шаг укрупнения кластеров.
* Методы построения дендрограммы:
    * Агломеративный метод (agglomerative) - объединение мелких кластеров
    * Дивизионный (дивизивный) метод (divisive) - деление крупных кластеров
* Методы расчета расстояния между кластерами
    * Метод одиночной связи (min расстояние)
    * Метод полной связи    (max расстояние)
    * Метод средней связи   (mean расстояние)
    * Центроидный метод     (расстояние м/ж центрами)
``` python
    from sklearn.cluster import AgglomerativeClustering
    agg_clst_model = AgglomerativeClustering()
```



### **ПОНИЖЕНИЕ РАЗМЕРНОСТИ**

#### СПЕКТРАЛЬНАЯ КЛАСТЕРИЗАЦИЯ
* 
* Применяется для: 
* сегментации изображений
``` python
    from sklearn.cluster import SpectralClustering
    spectral_clst_model = SpectralClustering()
```


#### PCA - линейное преобразование
* Метод главных компонент, или PCA (Principal Components Analysis)
* это один из базовых способов уменьшения размерности
* Применяется для:  
    * подавление шума
    * индексация видео
``` python
    from sklearn.decomposition import PCA
    pca = PCA()
```

#### t-SNE - нелинейное преобразование
* t-SNE (t-distributed Stochastic Neighbor Embedding)
* «стохастическое вложение соседей с t-распределением»
* при преобразовании похожие объекты оказываются рядом, а непохожие — далеко друг от друга
* Применяется для:  
    * уменьшения размерность до 2х или 3х мерного
``` python
    from sklearn.manifold import TSNE
    tsne = TSNE()
```

### КЛАСТЕРИЗАЦИЯ НА ОСНОВЕПЛОТНОСТИ (DBSCAN)
* DENSITY-BASED SPATIAL CLUSTERING OF APPLICATIONS WITH NOISE
* В отличие от k-means, не нужно задавать количество кластеров — алгоритм сам определит оптимальное
* Алгоритм хорошо работает с данными произвольной формы
* DBSCAN отлично справляется с выбросами в датасетах
``` python
    from sklearn.cluster import DBSCAN
    clst_model = DBSCAN()    
```


### ВИЗУАЛИЗАЦИЯ КЛАСТЕРИЗАЦИЙ
* диаграмма рассеяния для двухмерного и трёхмерного случаев 
* Convex Hull, или выпуклая оболочка - провести гарницы
* дендрограмма 
* Clustergram - "полосы" - только для иерархических кластеризаций

### МЕТРИКИ КЛАСТЕРИЗАЦИИ 
``` python
    from sklearn.metrics.cluster import homogeneity_score
    from sklearn.metrics.cluster import completeness_score
    from sklearn.metrics.cluster import v_measure_score
    from sklearn.metrics.cluster import rand_score
```
* Однородность кластеров                - homogeneity_score(y, y_pred)
* Полнота кластеров                     - completeness_score(y, y_pred)
* V-мера (комбинация однор. и полн.)    - v_measure_score(y, y_pred)
* Индекс Ренда                          - rand_score(y, y_pred)

# МЕТРИКИ КЛАССИФИКАЦИИ

**Accuracy** (достоверность) — доля правильных ответов модели среди всех ответов. 
* Интерпретация: как много (в долях) модель угадала ответов.
* Accuracy — самая простая и самая понятная метрика классификации, но у неё есть один существенный недостаток. 
* Она бесполезна, если классы сильно несбалансированы.

**Precision** (точность), или PPV (Positive Predictive Value) — это доля объектов, которые действительно являются положительными, по отношению ко всем объектам, названным моделью положительными.
* Интерпретация: способность отделить класс 1 от класса 0. Чем больше precision, тем меньше ложных попаданий. 
* Precision нужен в задачах, где от нас требуется минимум ложных срабатываний. Чем выше «цена» ложноположительного результата, тем выше должен быть precision.
* Можно использовать на несбалансированных выборках.

**Recall** (полнота), или TPR (True Positive Rate) — это доля объектов, названных классификатором положительными, по отношению ко всем объектам положительного класса.
* Интерпретация: способность модели обнаруживать класс 1 вообще, то есть охват класса 1. Заметьте, что ложные срабатывания не влияют на recall. 
* Recall очень хорошо себя показывает в задачах, где важно найти как можно больше объектов, принадлежащих к классу 1.
* Можно использовать на несбалансированных выборках.


Концентрация только на одной метрике (precision или recall) без учёта второй — сомнительная идея.
В битве за максимум precision для класса 1 побеждает модель, которая всегда будет говорить говорить «нет». У неё вообще не будет ложных срабатываний.
В битве за максимум recall для класса 1 побеждает модель, которая всегда будет говорить «да». Она охватит все наблюдения класса 1. 
В реальности необходимо балансировать между двумя этими метриками.

**F1** (F-мера) — это взвешенное среднее гармоническое между precision и recall:
* Несмотря на отсутствие бизнес-интерпретации, метрика F1 является довольно распространённой и используется в задачах, где необходимо выбрать модель, которая балансирует между precision и recall.
* Используется в задачах, где необходимо балансировать между precision и recall.

In [ ]:
#Модель log_reg_full:
#Рассчитываем accuracy
print('Accuracy: {:.2f}'.format(metrics.accuracy_score(y, y_pred)))
#Рассчитываем precision
print('Precision: {:.2f}'.format(metrics.precision_score(y, y_pred)))
#Рассчитываем recall
print('Recall: {:.2f}'.format(metrics.recall_score(y, y_pred)))
#Рассчитываем F1-меру
print('F1 score: {:.2f}'.format(metrics.f1_score(y, y_pred)))

In [ ]:
# Метрики классификации в одной строке
print(metrics.classification_report(y, y_pred))

In [ ]:
def plot_learning_curve(model, X, y, cv, scoring="f1", ax=None, title=""):
    # Вычисляем координаты для построения кривой обучения
    train_sizes, train_scores, valid_scores = model_selection.learning_curve(
        estimator=model,  # модель
        X=X,  # матрица наблюдений X
        y=y,  # вектор ответов y
        cv=cv,  # кросс-валидатор
        scoring=scoring,  # метрика
    )
    # Вычисляем среднее значение по фолдам для каждого набора данных
    train_scores_mean = np.mean(train_scores, axis=1)
    valid_scores_mean = np.mean(valid_scores, axis=1)
    # Если координатной плоскости не было передано, создаём новую
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 4))  # фигура + координатная плоскость
    # Строим кривую обучения по метрикам на тренировочных фолдах
    ax.plot(train_sizes, train_scores_mean, label="Train")
    # Строим кривую обучения по метрикам на валидационных фолдах
    ax.plot(train_sizes, valid_scores_mean, label="Valid")
    # Даём название графику и подписи осям
    ax.set_title("Learning curve: {}".format(title))
    ax.set_xlabel("Train data size")
    ax.set_ylabel("Score")
    # Устанавливаем отметки по оси абсцисс
    ax.xaxis.set_ticks(train_sizes)
    # Устанавливаем диапазон оси ординат
    ax.set_ylim(0, 1)
    # Отображаем легенду
    ax.legend()

Что отображено в print(metrics.classification_report(y, y_pred))?

1) В первой части таблицы отображаются метрики precision, recall и f1-score, рассчитанные для каждого класса в отдельности. Столбец support — это количество объектов каждого из классов.
2) Во второй части таблицы отображена общая метрика accuracy. 
3) Далее идёт строка macro avg — это среднее значение метрики между классами 1 и 0. Например, значение в строке macro avg и столбце recall = (0.88 + 0.56)/2=0.72.
4) Завершает отчёт строка weighted avg — это средневзвешенное значение метрики между классами 1 и 0. Рассчитывается по формуле:

In [ ]:
# Нас интересует только вероятность класса (второй столбец)
y_test_proba_pred = log_reg.predict_proba(X_test_scaled)[:, 1]
# Для удобства завернем numpy-массив в pandas Series
y_test_proba_pred = pd.Series(y_test_proba_pred)
# Создадим списки, в которых будем хранить значения метрик 
recall_scores = []
precision_scores = []
f1_scores = []
# Сгенерируем набор вероятностных порогов в диапазоне от 0.1 до 1
thresholds = np.arange(0.1, 1, 0.05)
# В цикле будем перебирать сгенерированные пороги
for threshold in thresholds:
    # Пациентов, для которых вероятность наличия диабета > threshold относим к классу 1
    # В противном случае - к классу 0
    y_test_pred = y_test_proba_pred.apply(lambda x: 1 if x>threshold else 0)
    # Считаем метрики и добавляем их в списки
    recall_scores.append(metrics.recall_score(y_test, y_test_pred))
    precision_scores.append(metrics.precision_score(y_test, y_test_pred))
    f1_scores.append(metrics.f1_score(y_test, y_test_pred))

# Визуализируем метрики при различных threshold
fig, ax = plt.subplots(figsize=(10, 4)) #фигура + координатная плоскость
# Строим линейный график зависимости recall от threshold
ax.plot(thresholds, recall_scores, label='Recall')
# Строим линейный график зависимости precision от threshold
ax.plot(thresholds, precision_scores, label='Precision')

# Строим линейный график зависимости F1 от threshold
ax.plot(thresholds, f1_scores, label='F1-score')
# Даем графику название и подписи осям
ax.set_title('Recall/Precision dependence on the threshold')
ax.set_xlabel('Probability threshold')
ax.set_ylabel('Score')
ax.legend();

In [ ]:
# Задаем оптимальный порог вероятностей
threshold_opt = 0.4 # полученное по графику значение
# Людей, у которых вероятность зарабатывать >50K больше 0.5 относим к классу 1
# В противном случае - к классу 0
y_test_pred_opt = y_test_proba_pred.apply(lambda x: 1 if x > threshold_opt else 0)
# Считаем метрики
print(metrics.classification_report(y_test, y_test_pred_opt))

# ЛОГИСТИЧЕСКАЯ РЕГРЕССИЯ В "КЛАССИФИКАЦИИ" С ПОМОЩЬЮ SKLEARN

In [ ]:
# Создаем объект класса логистическая регрессия
log_reg_full = linear_model.LogisticRegression(random_state=42, max_iter=1000)
# random_state — число, на основе которого происходит генерация случайных чисел.

# multi_class='multinomial' - мультиклассовая классификация

# penalty — метод регуляризации. Возможные значения:
#   'l1' — L1-регуляризация;
#   'l2' — L2-регуляризация (используется по умолчанию);
#   'elasticnet' — эластичная сетка (L1+L2);
#   'none' — отсутствие регуляризации.

# C — коэффициент обратный коэффициенту регуляризации, то есть равен C/α. 
# Чем больше C, тем меньше регуляризация. По умолчанию C=1, тогда α=1.

# solver — численный метод оптимизации функции потерь logloss, может быть:
#   'sag' — стохастический градиентный спуск (нужна стандартизация/нормализация);
#   'saga' — модификация предыдущего, которая поддерживает работу с негладкими функциями (нужна стандартизация/нормализация);
#   'newton-cg' — метод Ньютона с модификацией сопряжённых градиентов (не нужна стандартизация/нормализация);
#   'lbfgs' — метод Бройдена — Флетчера — Гольдфарба — Шанно (не нужна стандартизация/нормализация; используется по умолчанию, так как из всех методов теоретически обеспечивает наилучшую сходимость);
#   'liblinear' — метод покоординатного спуска (не нужна стандартизация/нормализация).

# max_iter — максимальное количество итераций, выделенных на сходимость.

# class_weight='balanced' - сбалансированность весов классов для дисбалансных классификаций

# Обучаем модель, минизируя logloss
log_reg_full.fit(X, y)

# Делаем предсказание класса
y_pred = log_reg_full.predict(X)

# Выводим результирующие коэффициенты
print('w0: {}'.format(log_reg_full.intercept_)) #свободный член w0
print('w1, w2: {}'.format(log_reg_full.coef_)) #остальные параметры модели w1, w2, ..., wm

# Значения концентации глюкозы и индекса массы тела для пациента
x_new = [[180, 51]]

# Делаем предсказание класса:
y_new_predict = log_reg_full.predict(x_new)
print('Predicted class: {}'.format(y_new_predict))

# Делаем предсказание вероятностей:
y_new_proba_predict = log_reg_full.predict_proba(x_new)
print('Predicted probabilities: {}'.format(np.round(y_new_proba_predict, 2)))

# Создадим временную таблицу X
X_temp = X.copy()
# Добавим в эту таблицу результат предсказания
X_temp['Prediction'] = y_pred
X_temp.tail()

# ДЕРЕВЬЯ РЕШЕНИЙ В "КЛАССИФИКАЦИИ" С ПОМОЩЬЮ SKLEARN

In [ ]:
from sklearn import tree # модели деревьев решения

# Создаем объект класса дерево решений
dt = tree.DecisionTreeClassifier(
    criterion='entropy',
    min_samples_leaf=5,
    max_depth=8,
    random_state=42
)

# criterion — критерий информативности ('gini' — критерий Джини и 'entropy' — энтропия Шеннона).

# max_depth — максимальная глубина дерева (по умолчанию — None, глубина дерева не ограничена).

# max_features — максимальное число признаков, по которым ищется лучшее разбиение в дереве 
# (по умолчанию — None, то есть обучение производится на всех признаках). Нужно потому, 
# что при большом количестве признаков будет «дорого» искать лучшее (по критерию типа прироста информации) 
# разбиение среди всех признаков.

# min_samples_leaf — минимальное число объектов в листе (по умолчанию — 1). 
# У этого параметра есть понятная интерпретация: если он равен 5, то дерево будет порождать 
# только те решающие правила, которые верны как минимум для пяти объектов.

# random_state — число, отвечающее за генерацию случайных чисел.

# Обучаем дерево по алгоритму CART
dt.fit(X_train, y_train)

# Выводим значения метрики 
y_train_pred = dt.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = dt.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Построение дерева на графике

In [ ]:
# Создаем фигуру для визуализации графа
fig = plt.figure(figsize=(25,20))
# Строим граф дерева решений
tree_graph = tree.plot_tree(
    dt, # объект обученного дерева
    feature_names=X_train.columns, # наименования факторов
    class_names=["0 - <=50K", "1 - >50K"], # имена классов
    filled=True, # расцветка графа
);

Построение графика с областями

In [ ]:
def plot_probabilities_2d(X, y, model):
    # Генерируем координатную сетку из всех возможных значений для признаков
    # Glucose изменяется от 40 до 200, BMI — от 10 до 80
    # Результат работы функции — два массива xx1 и xx2, которые образуют координатную сетку
    xx1, xx2 = np.meshgrid(
        np.arange(40, 200, 0.1),
        np.arange(10, 80, 0.1)
    )
    # Вытягиваем каждый из массивов в вектор-столбец — reshape(-1, 1)
    # Объединяем два столбца в таблицу с помощью hstack
    X_net = np.hstack([xx1.reshape(-1, 1), xx2.reshape(-1, 1)])
    # Предсказываем вероятность для всех точек на координатной сетке
    # Нам нужна только вероятность класса 1
    probs = model.predict_proba(X_net)[:, 1]
    # Переводим столбец из вероятностей в размер координатной сетки
    probs = probs.reshape(xx1.shape)
    # Создаём фигуру и координатную плоскость
    fig, ax = plt.subplots(figsize = (10, 5))
    # Рисуем тепловую карту вероятностей
    contour = ax.contourf(xx1, xx2, probs, 100, cmap='bwr')
    # Рисуем разделяющую плоскость — линию, где вероятность равна 0.5
    bound = ax.contour(xx1, xx2, probs, [0.5], linewidths=2, colors='black');
    # Добавляем цветовую панель 
    colorbar = fig.colorbar(contour)
    # Накладываем поверх тепловой карты диаграмму рассеяния
    sns.scatterplot(data=X, x='Glucose', y='BMI', hue=y, palette='seismic', ax=ax)
    # Даём графику название
    ax.set_title('Scatter Plot with Decision Boundary');
    # Смещаем легенду в верхний левый угол вне графика
    ax.legend(bbox_to_anchor=(-0.05, 1))

# Вызовем нашу функцию для визуализации:
plot_probabilities_2d(X, y, dt_clf_2d)

Важность признаков можно посмотреть, обратившись к атрибуту feature_importance_:

In [ ]:
print(dt_clf_full.feature_importances_)

# А лучше через столбчатую диаграмму

fig, ax = plt.subplots(figsize=(13, 5)) #фигура + координатная плоскость
feature = X.columns #признаки
feature_importances = dt_clf_full.feature_importances_ #важность признаков
# Строим столбчатую диаграмму
sns.barplot(x=feature, y=feature_importances, ax=ax);
# Добавляем подпись графику, осям абсцисс и ординат
ax.set_title('Bar plot feature importances')
ax.set_xlabel('Features')
ax.set_ylabel('Importances');

# АНСАМБЛИ

Ансамблевые модели или просто ансамбли (ensembles) — это метод машинного обучения, где несколько простых моделей (часто называемых «слабыми учениками») обучаются для решения одной и той же задачи и объединяются для получения лучших результатов.

Существует три проверенных способа построения ансамблей:

* Бэггинг — параллельно обучаем множество одинаковых моделей, а для предсказания берём среднее по предсказаниям каждой из моделей.
* Бустинг — последовательно обучаем множество одинаковых моделей, где каждая новая модель концентрируется на тех примерах, где предыдущая допустила ошибку.
* Стекинг — параллельно обучаем множество разных моделей, отправляем их результаты в финальную модель, и уже она принимает решение.

In [ ]:
from sklearn import ensemble #ансамбли

# СЛУЧАЙНЫЙ ЛЕС

**Случайный лес (Random Forest)** - частный случай бэггинга над деревьями решений с методом случайных подпространств

In [ ]:
# Создаём объект класса RandomForestClassifier
rf = ensemble.RandomForestClassifier(
    n_estimators=500, # число деревьев
    criterion='entropy', # критерий эффективности
    max_depth=3, # максимальная глубина дерева
    min_samples_leaf=10, # минимальное число объектов в листе
    max_features='sqrt', # число признаков из метода случайных подпространств
    random_state=42 # генератор случайных чисел
)

# n_estimators — количество деревьев в лесу (число K из бэггинга; по умолчанию равно 100);

# criterion — критерий информативности разбиения для каждого из деревьев 
# ('gini' — критерий Джини и 'entropy' — энтропия Шеннона; по умолчанию — 'gini');

# max_depth — максимальная глубина одного дерева (по умолчанию — None, то есть глубина дерева не ограничена);

# max_features — максимальное число признаков, которые будут использоваться каждым из деревьев 
# (число L из метода случайных подпространств; по умолчанию — 'sqrt'; 
# для обучения каждого из деревьев используется корень из m признаков, 
# где m — число признаков в начальном наборе данных);

# min_samples_leaf — минимальное число объектов в листе (по умолчанию — 1);

# random_state — параметр, отвечающий за генерацию случайных чисел.

# Обучаем модель 
rf.fit(X_train, y_train)
 
#Выводим значения метрики 
y_train_pred = rf.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = rf.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

In [ ]:
# Вызовем нашу функцию для визуализации:
plot_probabilities_2d(X, y, rf)

# Также можно показать важность признаков через столбчатую диаграмму

In [ ]:
y_test_proba_pred = pd.Series(rf.predict_proba(X_test)[:, 1])
#Создадим списки, в которых будем хранить значения метрик 
f1_scores = []
#Сгенерируем набор вероятностных порогов в диапазоне от 0.1 до 1
thresholds = np.arange(0.1, 1, 0.05)
#В цикле будем перебирать сгенерированные пороги
for threshold in thresholds:
    y_test_pred_poly = y_test_proba_pred.apply(lambda x: 1 if x > threshold else 0)
    #Считаем метрики и добавляем их в списки
    f1_scores.append(metrics.f1_score(y_test, y_test_pred_poly))

In [ ]:
#Визуализируем метрики при различных threshold
fig, ax = plt.subplots(figsize=(10, 4)) #фигура + координатная плоскость
#Строим линейный график зависимости F1 от threshold
ax.plot(thresholds, f1_scores, label='F1-score')

#Даем графику название и подписи осям
ax.set_title('F1 dependence on the threshold')
ax.set_xlabel('Probability threshold')
ax.set_ylabel('Score')
#Устанавливаем отметки по оси x
ax.set_xticks(thresholds) 
# Подписываем названия осей
ax.set_xlabel('Probability threshold')
ax.set_ylabel('Score')
#Отображаем легенду
ax.legend();

# КЛАСТЕРИЗАЦИЯ

# МЕТРИКИ КЛАСТЕРИЗАЦИИ

In [ ]:
# импортируем подсчет метрики ОДНОРОДНОСИТ кластеров
from sklearn.metrics.cluster import homogeneity_score

# передаем предсказанную информацию к какому кластеру относятся объекты датасета и правильные ответы
print(homogeneity_score(labels_true=[0, 0, 1, 1], labels_pred=[0, 0, 1, 1]))

# теперь посчитаем насколько однородными получились кластеры с покемонами
print(homogeneity_score(labels_true=df.RealClusters, labels_pred=df.Clusters_k3))

print(homogeneity_score(labels_true=df.RealClusters, labels_pred=df.Clusters_k4))

In [ ]:
# импортируем подсчет метрики ПОЛНОТЫ кластеров
from sklearn.metrics.cluster import completeness_score

# передаем предсказанную информацию к какому кластеру относятся объекты датасета и правильные ответы, подсчитываем метрику
print(completeness_score(labels_true=[0, 0, 1, 1], labels_pred=[0, 0, 1, 1]))

# посчитаем насколько полными получились кластеры с покемонами
print(completeness_score(labels_true=df.RealClusters, labels_pred=df.Clusters_k3))

# посчитаем насколько полными получились кластеры с покемонами
print(completeness_score(labels_true=df.RealClusters, labels_pred=df.Clusters_k4))

In [ ]:
# импортируем из библиотеки sklearn подсчет V-меры
from sklearn.metrics import v_measure_score
# Эта метрика — комбинация метрик полноты и однородности.

# теперь посчитаем v-меру для кластеров с покемонами
print(v_measure_score(labels_true=df.RealClusters, labels_pred=df.Clusters_k3))

print(v_measure_score(labels_true=df.RealClusters, labels_pred=df.Clusters_k4))

In [ ]:
# импортируем из библиотеки sklearn подсчет ИНДЕКСА РЭНДА
from sklearn.metrics.cluster import rand_score

# теперь посчитаем насколько полными получились кластеры с покемонами
print(rand_score(labels_true=df.RealClusters, labels_pred=df.Clusters_k3))

print(rand_score(labels_true=df.RealClusters, labels_pred=df.Clusters_k4))